In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200) 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import resample
# Import metrics used
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('../data/california_wildfires.csv')

In [3]:
df

,Unnamed: 0,date,county,year,acres_burned,fire_started,Unnamed: 0.1,Alfalfa & Hay_acres,Alfalfa & Hay_percentage,Almonds_acres,Almonds_percentage,Barren_acres,Barren_percentage,Corn_acres,Corn_percentage,Cotton_acres,Cotton_percentage,Deciduous Forest_acres,Deciduous Forest_percentage,Evergreen Forest_acres,Evergreen Forest_percentage,Fallow_acres,Fallow_percentage,Fruit Trees_acres,Fruit Trees_percentage,Grain Crops_acres,Grain Crops_percentage,Grapes_acres,Grapes_percentage,Grassland_acres,Grassland_percentage,High Intensity Developed_acres,High Intensity Developed_percentage,Low Intensity Developed_acres,Low Intensity Developed_percentage,Mixed Forest_acres,Mixed Forest_percentage,Other Ocean/Mexico_acres,Other Ocean/Mexico_percentage,Other Tree Crops_acres,Other Tree Crops_percentage,Other_acres,Other_percentage,Rice_acres,Rice_percentage,Shrubland_acres,Shrubland_percentage,Tomatoes_acres,Tomatoes_percentage,Vegs & Fruits_acres,Vegs & Fruits_percentage,Walnuts_acres,Walnuts_percentage,Water_acres,Water_percentage,Wetlands_acres,Wetlands_percentage,Winter Wheat_acres,Winter Wheat_percentage,max_elevation,min_elevation,Avg Air Temp (F)_Weekly,Avg Rel Hum (%)_Weekly,Avg Wind Speed (mph)_Weekly,Dew Point (F)_Weekly,Max Air Temp (F)_Weekly,Max Rel Hum (%)_Weekly,Min Air Temp (F)_Weekly,Min Rel Hum (%)_Weekly,Precip (in)_Weekly,Avg Air Temp (F)_month,Avg Rel Hum (%)_month,Avg Wind Speed (mph)_month,Dew Point (F)_month,Max Air Temp (F)_month,Max Rel Hum (%)_month,Min Air Temp (F)_month,Min Rel Hum (%)_month,Precip (in)_month,Population,county_acres,pop_density
0,0,2013-01-06,Alameda,2013,0.0,0.0,2,1102.856805,0.300074,4.225505,0.001150,194.595625,0.052947,4.670295,0.001271,0.000000,0.000000,5.337480,0.001452,7838.756565,2.132827,1536.749450,0.418130,1.779160,0.000484,991.214515,0.269697,3722.447510,1.012831,153671.386680,41.812059,28431.421590,7.735834,39470.886995,10.739534,74885.956375,20.375531,0.0,0.0,8.673405,0.002360,0.0,0.0,0.889580,0.000242,30958.051185,8.423298,4.670295,0.001271,164.127510,0.044657,4.670295,0.001271,19403.518960,5.279454,4497.494085,1.223712,624.485160,0.169915,1242,-42,44.214286,82.785714,2.392857,39.321429,54.157143,96.500000,35.771429,60.785714,0.095714,45.506897,78.189655,2.915517,38.932759,55.896552,95.448276,35.725862,55.810345,0.130172,1567167,528000,2.968119
1,1,2013-01-06,Alpine,2013,0.0,0.0,2,189.035750,0.040080,0.000000,0.000000,15482.472715,3.282650,0.000000,0.000000,0.000000,0.000000,194.595625,0.041259,195088.007530,41.363269,0.444790,0.000094,0.222395,0.000047,0.000000,0.000000,0.000000,0.000000,5644.829890,1.196837,121.427670,0.025746,3192.480225,0.676881,0.667185,0.000141,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,247783.390805,52.535935,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2650.503610,0.561969,1297.452430,0.275091,0.000000,0.000000,3556,1442,29.657143,76.514286,3.228571,21.328571,34.428571,91.857143,22.857143,55.428571,0.000000,30.789655,68.162069,4.968966,19.600000,39.344828,86.000000,22.758621,46.344828,0.000000,1164,465280,0.002502
2,2,2013-01-06,Amador,2013,0.0,0.0,2,1326.808570,0.414290,16.679625,0.005208,1873.010690,0.584840,242.632945,0.075761,0.000000,0.000000,17190.911105,5.367789,114386.866695,35.716810,168.130620,0.052498,12.009330,0.003750,120.093300,0.037499,2587.343430,0.807887,112912.610240,35.256480,440.119705,0.137425,8263.975805,2.580391,1727.119570,0.539286,0.0,0.0,1.334370,0.000417,0.0,0.0,1.111975,0.000347,52457.865415,16.379744,0.000000,0.000000,1.779160,0.000556,122.094855,0.038124,5822.745890,1.818128,105.860020,0.033054,479.483620,0.149717,3121,43,34.114286,83.571429,3.157143,29.585714,40.071429,96.000000,27.757143,66.571429,0.141429,34.289655,76.724138,3.606897,27.410345,41.200000,93.172414,27.768966,58.310345,0.155517,37304,384640,0.096984
3,3,2013-01-06,Butte,2013,0.0,0.0,2,3777.156680,0.374865,46196.556585,4.584787,1869.452370,0.185534,2023.349710,0.200808,9.118195,0.000905,33181.556395,3.293111,408193.790775,40.511281,56434.510410

In [4]:
# Create dummy variables for the county column
counties = pd.get_dummies(df.county, drop_first = True)
# Drop county column along with unnecessary columns (Unnamed columns, year, and acres burned)
df2 = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'county', 'year', 'acres_burned'], axis = 1)

In [5]:
# Feature engineer month column from the date column
df2['month'] = pd.DatetimeIndex(df2['date']).month
# Drop the date column
df2.drop(columns = ['date'], axis = 1, inplace = True)

In [6]:
# Create dummy variables for the months
month = pd.get_dummies(df2.month, drop_first = True)
# Drop the month column
df2.drop(columns = 'month', axis =1, inplace = True)

In [7]:
# Combine the original dataframe with the dummy variables
df2 = pd.concat([df2, counties, month], axis = 1)

In [8]:
# Split dataset into target variable and features
y = df2.fire_started
X = df2.drop(columns = ['fire_started'], axis = 1)

In [9]:
# Use train_test_split to create a training dataset and test dataset
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size = 0.25, random_state = 0)

In [10]:
# Concatenate the X_train and y_train back into one training dataframe
training = pd.concat([X_train, y_train], axis=1)

In [11]:
# Split the target variable by class into two dataframes
no_fire = training[training.fire_started == 0] # 0 = No Wildfire
fire = training[training.fire_started == 1] # 1 = Wildfire

In [12]:
# Resample the minority class (wildfire)
fire_upsampled = resample(fire,
                          replace=True, # sample with replacement
                          n_samples=no_fire.shape[0], # match number in majority class
                          random_state=42) # reproducible result

In [13]:
# Create a new dataframe combining the target classes
resampled_df = pd.concat([no_fire, fire_upsampled])

In [14]:
# Double-check the different target classes
resampled_df.fire_started.value_counts()

1.0    12812
0.0    12812
Name: fire_started, dtype: int64

In [15]:
y_train = resampled_df.fire_started
X_train = resampled_df.drop(columns = ['fire_started'], axis = 1)

## Logistic Regression

### Base Model

In [16]:
# Instantiate a logistic regression model
logreg = LogisticRegression(random_state = 0) # random state for consistant results
# Train model on resampled training data
logreg.fit(X_train, y_train)

/Users/malcolmkatzenbach/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=0)

In [17]:
# Use the model to predict the target variable on the training dataset
y_hat_train = logreg.predict(X_train)
# Use the model to predict the target variable on the test dataset
y_hat_test = logreg.predict(X_test)

In [18]:
# Print the f1 score metric on both the training and test predictions to check for overfitting
print('F1 Score: Training', f1_score(y_train, y_hat_train), 'Test', f1_score(y_test, y_hat_test))
# Print the accuracy score metric on both the training and test predictions to check for overfitting
print('Accuracy Score: Training', accuracy_score(y_train, y_hat_train), 'Test', accuracy_score(y_test, y_hat_test))
# Print the recall score metric on both the training and test predictions to check for overfitting
print('Recall Score: Training', recall_score(y_train, y_hat_train), 'Test', recall_score(y_test, y_hat_test))

F1 Score: Training 0.6259625586091242 Test 0.1420159334949775
Accuracy Score: Training 0.5734857945675929 Test 0.45428508482044505
Recall Score: Training 0.7137839525444896 Test 0.7068965517241379


In [19]:
confusion_matrix(y_test, y_hat_test)

array([[1857, 2392],
       [  85,  205]])

### Scaled Data

In [64]:
# Insantiate the StandardScaler()
ss = StandardScaler()
# Fit the feature training data
ss.fit(X_train)

# Transform both the training and test features
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

In [65]:
# Instantiate a new logistic regression model
logreg1 = LogisticRegression(solver = 'liblinear')
# Fit the data to the new scaled data
logreg1.fit(X_train_scaled, y_train)

LogisticRegression(solver='liblinear')

In [66]:
# Use model to predict target variable on the training dataset
y_hat_train1 = logreg1.predict(X_train_scaled)
# Use model to predict target variable on the test dataset
y_hat_test1 = logreg1.predict(X_test_scaled)

In [67]:
# Print the f1 score metric on both the training and test predictions to check for overfitting
print('F1 Score: Training', f1_score(y_train, y_hat_train1), 'Test', f1_score(y_test, y_hat_test1))
# Print the accuracy score metric on both the training and test predictions to check for overfitting
print('Accuracy Score: Training', accuracy_score(y_train, y_hat_train1), 'Test', accuracy_score(y_test, y_hat_test1))
# Print the recall score metric on both the training and test predictions to check for overfitting
print('Recall Score: Training', recall_score(y_train, y_hat_train1), 'Test', recall_score(y_test, y_hat_test1))

F1 Score: Training 0.8157728940443985 Test 0.28536436007348437
Accuracy Score: Training 0.8060021854511396 Test 0.7428949107732981
Recall Score: Training 0.8590384014985951 Test 0.803448275862069


In [68]:
confusion_matrix(y_test, y_hat_test1)

array([[3139, 1110],
       [  57,  233]])

In [69]:
scores = cross_val_score(logreg1, X_train_scaled, y_train, cv=10)
print(scores.mean())

0.8017092607432437


In [84]:
predicted = pd.concat([X_test, y_test], axis = 1)
predicted

,Alfalfa & Hay_acres,Alfalfa & Hay_percentage,Almonds_acres,Almonds_percentage,Barren_acres,Barren_percentage,Corn_acres,Corn_percentage,Cotton_acres,Cotton_percentage,Deciduous Forest_acres,Deciduous Forest_percentage,Evergreen Forest_acres,Evergreen Forest_percentage,Fallow_acres,Fallow_percentage,Fruit Trees_acres,Fruit Trees_percentage,Grain Crops_acres,Grain Crops_percentage,Grapes_acres,Grapes_percentage,Grassland_acres,Grassland_percentage,High Intensity Developed_acres,High Intensity Developed_percentage,Low Intensity Developed_acres,Low Intensity Developed_percentage,Mixed Forest_acres,Mixed Forest_percentage,Other Ocean/Mexico_acres,Other Ocean/Mexico_percentage,Other Tree Crops_acres,Other Tree Crops_percentage,Other_acres,Other_percentage,Rice_acres,Rice_percentage,Shrubland_acres,Shrubland_percentage,Tomatoes_acres,Tomatoes_percentage,Vegs & Fruits_acres,Vegs & Fruits_percentage,Walnuts_acres,Walnuts_percentage,Water_acres,Water_percentage,Wetlands_acres,Wetlands_percentage,Winter Wheat_acres,Winter Wheat_percentage,max_elevation,min_elevation,Avg Air Temp (F)_Weekly,Avg Rel Hum (%)_Weekly,Avg Wind Speed (mph)_Weekly,Dew Point (F)_Weekly,Max Air Temp (F)_Weekly,Max Rel Hum (%)_Weekly,Min Air Temp (F)_Weekly,Min Rel Hum (%)_Weekly,Precip (in)_Weekly,Avg Air Temp (F)_month,Avg Rel Hum (%)_month,Avg Wind Speed (mph)_month,Dew Point (F)_month,Max Air Temp (F)_month,Max Rel Hum (%)_month,Min Air Temp (F)_month,Min Rel Hum (%)_month,Precip (in)_month,Population,county_acres,pop_density,Alpine,Amador,Butte,Calaveras,Colusa,Contra Costa,Del Norte,El Dorado,Fresno,Glenn,Humboldt,Imperial,Inyo,Kern,Kings,Lake,Lassen,Los Angeles,Madera,Marin,Mariposa,Mendocino,Merced,Modoc,Mono,Monterey,Napa,Nevada,Orange,Placer,Plumas,Riverside,Sacramento,San Benito,San Bernardino,San Diego,San Francisco,San Joaquin,San Luis Obispo,San Mateo,Santa Barbara,Santa Clara,Santa Cruz,Shasta,Sierra,Siskiyou,Solano,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba,2,3,4,5,6,7,8,9,10,11,12,fire_started
12930,17.791600,0.001264,3.780715,0.000269,120475.152215,8.559617,0.000000,0.000000,0.000000,0.000000,17857.428920,1.268749,673694.279255,47.865180,9.562985,0.000679,1.779160,0.000126,1.556765,0.000111,0.889580,0.000063,107346.063390,7.626811,556.432290,0.039534,11897.687710,0.845317,265.094840,0.018835,0.0,0.0,1.556765,0.000111,0.0,0.0,0.000000,0.000000,4.451974e+05,31.630749,0.000000,0.000000,0.000000,0.000000,0.222395,0.000016,29309.214655,2.082385,816.189650,0.057989,30.912905,0.002196,3959,374,60.528571,60.714286,11.685714,44.942857,73.714286,88.285714,48.000000,33.000000,0.004286,59.082759,70.362069,7.817241,47.824138,70.793103,93.379310,48.689655,41.827586,0.098276,54725,1467520,0.037291,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0
17287,5199.595100,0.516035,36487.680465,3.621228,2509.060390,0.249012,348.270570,0.034564,221.505420,0.021983,36955.154755,3.667622,389571.545450,38.663112,82134.254215,8.151432,19776.252980,1.962698,919.380930,0.091244,42.477445,0.004216,148690.405865,14.756811,5127.316725,0.508862,28198.796420,2.798596,583.786875,0.057938,0.0,0.0,2598.240785,0.257863,0.0,0.0,79618.521975,7.901757,8.180177e+04,8.118435,777.715315,0.077185,516.401190,0.051250,51968.374020,5.157612,21834.741100,2.166994,11160.893075,1.107665,563.104140,0.055885,2192,-1,64.800000,61.142857,2.242857,51.114286,81.900000,90.714286,50.757143,31.285714,0.000000,68.113793,61.172414,2.162069,54.089655,86.786207,92.413793,52.775862,32.620690,0.000000,227621,1065600,0.213608,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0
11545,3504.945200,0.347849,43919.676575,4.358818,2064.937575,0.204935,270.209925,0.026817,81.396570,0.008078,36126.733380,3.585405,391415.422395,38.846108,63940.786450,6.345817,8684.302355,0.861875,568.886410,0.056459,393.416755,0.03904

In [86]:
predicted.loc[predicted.fire_started != y_hat_test1].describe()

,Alfalfa & Hay_acres,Alfalfa & Hay_percentage,Almonds_acres,Almonds_percentage,Barren_acres,Barren_percentage,Corn_acres,Corn_percentage,Cotton_acres,Cotton_percentage,Deciduous Forest_acres,Deciduous Forest_percentage,Evergreen Forest_acres,Evergreen Forest_percentage,Fallow_acres,Fallow_percentage,Fruit Trees_acres,Fruit Trees_percentage,Grain Crops_acres,Grain Crops_percentage,Grapes_acres,Grapes_percentage,Grassland_acres,Grassland_percentage,High Intensity Developed_acres,High Intensity Developed_percentage,Low Intensity Developed_acres,Low Intensity Developed_percentage,Mixed Forest_acres,Mixed Forest_percentage,Other Ocean/Mexico_acres,Other Ocean/Mexico_percentage,Other Tree Crops_acres,Other Tree Crops_percentage,Other_acres,Other_percentage,Rice_acres,Rice_percentage,Shrubland_acres,Shrubland_percentage,Tomatoes_acres,Tomatoes_percentage,Vegs & Fruits_acres,Vegs & Fruits_percentage,Walnuts_acres,Walnuts_percentage,Water_acres,Water_percentage,Wetlands_acres,Wetlands_percentage,Winter Wheat_acres,Winter Wheat_percentage,max_elevation,min_elevation,Avg Air Temp (F)_Weekly,Avg Rel Hum (%)_Weekly,Avg Wind Speed (mph)_Weekly,Dew Point (F)_Weekly,Max Air Temp (F)_Weekly,Max Rel Hum (%)_Weekly,Min Air Temp (F)_Weekly,Min Rel Hum (%)_Weekly,Precip (in)_Weekly,Avg Air Temp (F)_month,Avg Rel Hum (%)_month,Avg Wind Speed (mph)_month,Dew Point (F)_month,Max Air Temp (F)_month,Max Rel Hum (%)_month,Min Air Temp (F)_month,Min Rel Hum (%)_month,Precip (in)_month,Population,county_acres,pop_density,Alpine,Amador,Butte,Calaveras,Colusa,Contra Costa,Del Norte,El Dorado,Fresno,Glenn,Humboldt,Imperial,Inyo,Kern,Kings,Lake,Lassen,Los Angeles,Madera,Marin,Mariposa,Mendocino,Merced,Modoc,Mono,Monterey,Napa,Nevada,Orange,Placer,Plumas,Riverside,Sacramento,San Benito,San Bernardino,San Diego,San Francisco,San Joaquin,San Luis Obispo,San Mateo,Santa Barbara,Santa Clara,Santa Cruz,Shasta,Sierra,Siskiyou,Solano,Sonoma,Stanislaus,Sutter,Tehama,Trinity,Tulare,Tuolumne,Ventura,Yolo,Yuba,2,3,4,5,6,7,8,9,10,11,12,fire_started
count,1167.000000,1167.000000,1167.000000,1167.000000,1.167000e+03,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1.167000e+03,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1.167000e+03,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1.167000e+03,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1.167000e+03,1.167000e+03,1167.000000,1167.0,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.0,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.0,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000
mean,23313.990675,1.561762,22064.063504,1.371046,1.075054e+05,2.498277,2427.321938,0.315772,4184.554972,0.215082,9766.566374,0.924501,4.356794e+05,24.284078,25891.105460,1.596966,7149.825641,0.350702,11451.757340,0.843314,18699.079653,1

## KNN

### Base Model

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [27]:
# Instantiate a knn model using 3 nearest neighbors
knn = KNeighborsClassifier(n_neighbors = 3)

In [28]:
# Fit knn model using the scaled data from the previous scaled logistic model
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [29]:
# Predict target variable for both the train and test datasets.
knn_train = knn.predict(X_train_scaled)
knn_test = knn.predict(X_test_scaled)

In [30]:
# Print the f1 score metric on both the training and test predictions to check for overfitting
print('F1 Score: Training', f1_score(y_train, knn_train), 'Test', f1_score(y_test, knn_test))
# Print the sccuracy score metric on both the training and test predictions to check for overfitting
print('Accuracy Score: Training', accuracy_score(y_train, knn_train), 'Test', accuracy_score(y_test, knn_test))
# Print the recall score metric on both the training and test predictions to check for overfitting
print('Recall Score: Training', recall_score(y_train, knn_train), 'Test', recall_score(y_test, knn_test))

F1 Score: Training 0.975706343766659 Test 0.2788339670468948
Accuracy Score: Training 0.9751014673743366 Test 0.874641991628112
Recall Score: Training 1.0 Test 0.3793103448275862


### Tuned

In [31]:
# Create a function to find the max recall score and return the score along with the k value
def max_value(l):
    max_val = max(l)
    max_idx = l.index(max_val)
    return max_idx, max_val

In [32]:
# Create an empty list for recall scores
k_scores = []
# Choose a range of k values to test
k_range = list(range(1, 21))
# Iterate through the different k values
for k in k_range:
    # Instantiate new knn model with k nearest neighbors
    knn = KNeighborsClassifier(n_neighbors = k)
    # Fit knn model on scaled training data
    knn.fit(X_train_scaled, y_train)
    # Use model to predict target variable on testing set
    y_pred = knn.predict(X_test_scaled)
    # Find the recall score
    recall = recall_score(y_test, y_pred)
    # Append recall score to list of recall scorees
    k_scores.append(recall)

# Find max recall score
idx, val = max_value(k_scores)
# Print max recall score and it corresponding k value
print(idx + 1, val)

19 0.7413793103448276


In [33]:
knn = KNeighborsClassifier(n_neighbors = 19)
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
knn_train1 = knn.predict(X_train_scaled)
print('F1 Score: Training', f1_score(y_train, knn_train1), 'Test', f1_score(y_test, y_pred))
print('Accuracy Score: Training', accuracy_score(y_train, knn_train1), 'Test', accuracy_score(y_test, y_pred))
print('Recall Score: Training', recall_score(y_train, knn_train1), 'Test', recall_score(y_test, y_pred))

F1 Score: Training 0.883275080458179 Test 0.24826789838337182
Accuracy Score: Training 0.8683655947549173 Test 0.7131526768010575
Recall Score: Training 0.9960974086793631 Test 0.7413793103448276


## Decision Tree

### Base Model

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
dt = DecisionTreeClassifier(random_state = 0)
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [36]:
dt_train = dt.predict(X_train)
dt_test = dt.predict(X_test)

print('F1 Score: Training', f1_score(y_train, dt_train), 'Test', f1_score(y_test, dt_test))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train), 'Test', accuracy_score(y_test, dt_test))
print('Recall Score: Training', recall_score(y_train, dt_train), 'Test', recall_score(y_test, dt_test))

F1 Score: Training 1.0 Test 0.23693379790940766
Accuracy Score: Training 1.0 Test 0.9035029742233972
Recall Score: Training 1.0 Test 0.23448275862068965


### GridSearchCV

In [37]:
from sklearn.model_selection import GridSearchCV

In [40]:
parameters = {'max_depth' : range(1, 15, 1), 'max_features' : range(55, 75, 1), 'min_samples_split' : range(10, 20, 1)}

In [41]:
dtg = DecisionTreeClassifier(random_state = 0)
grid_model = GridSearchCV(dtg, parameters, cv = 10, scoring = 'f1', verbose = 1, n_jobs = -1)

grid_model.fit(X_train, y_train)

Fitting 10 folds for each of 2800 candidates, totalling 28000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed:   52.3s
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4968 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 6018 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 7168 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 8418 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 9768 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 11218 tasks      

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': range(1, 15),
                         'max_features': range(55, 75),
                         'min_samples_split': range(10, 20)},
             scoring='f1', verbose=1)

In [42]:
print(grid_model.best_score_)
print(grid_model.best_params_)
print(grid_model.best_estimator_)

0.9242561036170699
{'max_depth': 14, 'max_features': 62, 'min_samples_split': 17}
DecisionTreeClassifier(max_depth=14, max_features=62, min_samples_split=17,
                       random_state=0)


In [43]:
dt_train2 = dt.predict(X_train)
dt_test2 = dt.predict(X_test)

print('F1 Score: Training', f1_score(y_train, dt_train2), 'Test', f1_score(y_test, dt_test2))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train2), 'Test', accuracy_score(y_test, dt_test2))
print('Recall Score: Training', recall_score(y_train, dt_train2), 'Test', recall_score(y_test, dt_test2))

F1 Score: Training 1.0 Test 0.23693379790940766
Accuracy Score: Training 1.0 Test 0.9035029742233972
Recall Score: Training 1.0 Test 0.23448275862068965


In [44]:
confusion_matrix(y_test, dt_test2)

array([[4033,  216],
       [ 222,   68]])

### GridSearchCV 2

In [45]:
parameters2 = {'max_depth' : range(10, 13, 1), 'max_features' : range(35, 65, 1), 'min_samples_split' : range(14, 30, 1)}

In [46]:
dtg2 = DecisionTreeClassifier(random_state = 0)
dtg2_model = GridSearchCV(dtg2, parameters2, cv = 10, scoring = 'f1', verbose = 1, n_jobs = -1)

dtg2_model.fit(X_train, y_train)

Fitting 10 folds for each of 1440 candidates, totalling 14400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 4968 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 6018 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 7168 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 8418 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 9768 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 11218 tasks      

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': range(10, 13),
                         'max_features': range(35, 65),
                         'min_samples_split': range(14, 30)},
             scoring='f1', verbose=1)

In [47]:
print(dtg2_model.best_score_)
print(dtg2_model.best_params_)
print(dtg2_model.best_estimator_)

0.9151798193645881
{'max_depth': 12, 'max_features': 52, 'min_samples_split': 15}
DecisionTreeClassifier(max_depth=12, max_features=52, min_samples_split=15,
                       random_state=0)


In [48]:
dt_train3 = dtg2_model.predict(X_train)
dt_test3 = dtg2_model.predict(X_test)

print('F1 Score: Training', f1_score(y_train, dt_train3), 'Test', f1_score(y_test, dt_test3))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train3), 'Test', accuracy_score(y_test, dt_test3))
print('Recall Score: Training', recall_score(y_train, dt_train3), 'Test', recall_score(y_test, dt_test3))

F1 Score: Training 0.9134525605113841 Test 0.26426426426426425
Accuracy Score: Training 0.9059475491726506 Test 0.7840934126459572
Recall Score: Training 0.9926631283172026 Test 0.6068965517241379


### GridSearch 3

In [95]:
parameters3 = {'max_depth' : range(8, 9, 1), 'max_features' : range(35, 65, 1), 'min_samples_split' : range(14, 30, 1)}

In [96]:
dtg3 = DecisionTreeClassifier(random_state = 0)
dtg3_model = GridSearchCV(dtg3, parameters3, cv = 10, scoring = 'f1', verbose = 1, n_jobs = -1)

dtg3_model.fit(X_train, y_train)

Fitting 10 folds for each of 480 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 1504 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 2404 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 3504 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 4658 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed:  1.9min finished


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': range(8, 9),
                         'max_features': range(35, 65),
                         'min_samples_split': range(14, 30)},
             scoring='f1', verbose=1)

In [97]:
print(dtg3_model.best_score_)
print(dtg3_model.best_params_)
print(dtg3_model.best_estimator_)

0.8665665288129925
{'max_depth': 8, 'max_features': 42, 'min_samples_split': 20}
DecisionTreeClassifier(max_depth=8, max_features=42, min_samples_split=20,
                       random_state=0)


In [98]:
dt_train4 = dtg3_model.predict(X_train)
dt_test4 = dtg3_model.predict(X_test)

print('F1 Score: Training', f1_score(y_train, dt_train4), 'Test', f1_score(y_test, dt_test4))
print('Accuracy Score: Training', accuracy_score(y_train, dt_train4), 'Test', accuracy_score(y_test, dt_test4))
print('Recall Score: Training', recall_score(y_train, dt_train4), 'Test', recall_score(y_test, dt_test4))

F1 Score: Training 0.8702843468468469 Test 0.2608173076923077
Accuracy Score: Training 0.8561504839213238 Test 0.7290152015862524
Recall Score: Training 0.9651108335935061 Test 0.7482758620689656
